In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

save_path = "/content/drive/MyDrive/gita_project/"
os.makedirs(save_path, exist_ok=True)  # This will create the folder if it doesn't exist


In [ ]:
# === 1. Install dependencies ===
!pip install -q streamlit streamlit-chat pyngrok faiss-cpu sentence-transformers cohere


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211

In [ ]:
# === 2. Write model_script.py ===
model_script = '''
import os
import json
import numpy as np
import faiss
import re
import cohere
from sentence_transformers import SentenceTransformer

load_path = "/content/drive/MyDrive/gita_project/"
index = faiss.read_index(load_path + "gita_index.faiss")

with open(load_path + "gita_texts.json", "r", encoding="utf-8") as f:
    texts = json.load(f)

with open(load_path + "gita_records.json", "r", encoding="utf-8") as f:
    records = json.load(f)

with open(load_path + "chapter_summaries.json", "r", encoding="utf-8") as f:
    chapter_summaries_raw = json.load(f)

chapter_summaries = {int(k): v for k, v in chapter_summaries_raw.items()}
id_map = {i: records[i] for i in range(len(records))}
model = SentenceTransformer("all-MiniLM-L6-v2")

os.environ["COHERE_API_KEY"] = "j92ZwLSyxUJzU80UHBbldJ7gUVodVqkmYQaEcqjS"
co = cohere.Client(os.getenv("COHERE_API_KEY"))

def detect_query_type(query):
    query = query.lower()
    if any(kw in query for kw in ["word meaning", "explain word", "शब्द", "शब्दार्थ"]):
        return "word_meaning"
    elif any(kw in query for kw in ["hindi", "अर्थ", "मतलब", "हिंदी"]):
        return "hindi_meaning"
    elif any(kw in query for kw in ["shloka", "sanskrit", "original", "श्लोक"]):
        return "shloka"
    elif any(kw in query for kw in ["summary", "message of chapter", "what is chapter", "explain chapter"]):
        return "chapter_summary"
    elif any(kw in query for kw in ["list chapters", "chapter names", "all chapters", "what are the chapters", "names of chapters"]):
        return "list_chapters"
    else:
        return "english_meaning"

def extract_verse_reference(query):
    match = re.search(r'(\\d+)\\.(\\d+)', query)
    if match:
        return f"{match.group(1)}.{match.group(2)}"
    return None

def extract_chapter_number(query):
    match = re.search(r'(chapter|adhyaya)\\s*(\\d+)', query.lower())
    if match:
        return int(match.group(2))
    return None

def generate_final_answer(query, retrieved_text):
    prompt = f"""You are a helpful assistant that answers questions based on the Bhagavad Gita.
Answer the following user question using the retrieved text.

User question: {query}

Retrieved data:
{retrieved_text}

Answer:"""

    response = co.chat(
        model="command-nightly",
        message=prompt,
        temperature=0.7,
        max_tokens=1500
    )
    return response.text.strip()

def search_verses(query, top_k=3):
    query_type = detect_query_type(query)

    if query_type == "chapter_summary":
        chapter_num = extract_chapter_number(query)
        if chapter_num and chapter_num in chapter_summaries:
            summary_data = chapter_summaries[chapter_num]
            title = summary_data.get("title", f"Chapter {chapter_num}")
            summary = summary_data.get("summary", "No summary available.")
            retrieved_text = f"🕉️ **Chapter {chapter_num}: {title}**\\n📝 **Summary**: {summary}"
            return generate_final_answer(query, retrieved_text)
        else:
            return f"❌ No summary found for chapter {chapter_num}."

    verse_id = extract_verse_reference(query)

    if query_type == "list_chapters":
        chapter_lines = []
        for chapter_num in sorted(chapter_summaries):
            chapter = chapter_summaries[chapter_num]
            title = chapter.get("title", f"Chapter {chapter_num}")
            chapter_lines.append(f"{chapter_num}. {title}")
        retrieved_text = "🕉️ **Chapters in the Bhagavad Gita**:\\n\\n" + "\\n".join(chapter_lines)
        return generate_final_answer(query, retrieved_text)

    if verse_id:
        results = [rec for rec in records if rec['id'] == verse_id]
        if not results:
            return f"❌ No verse found for ID {verse_id}"
    else:
        query_embedding = model.encode([query], convert_to_numpy=True)
        D, I = index.search(np.array(query_embedding), top_k)
        results = [id_map[i] for i in I[0]]

    output_blocks = []
    for res in results:
        vid = res['id']
        context = res.get('context', '')
        meta = res.get('metadata', {})
        answer = meta.get(query_type) or "❌ No data available for this query type."
        shloka = meta.get("transliteration", "")
        eng = res.get("text", "").split("Commentary")[0].strip()
        commentary = meta.get("commentary", "")

        block = f"""📖 **Verse:** {vid} ({context})\\n🕉️ **Shloka**: {shloka}\\n💬 **Answer**: {answer}\\n📘 **English Meaning**: {eng}\\n🧠 **Commentary**: {commentary}"""
        output_blocks.append(block)

    retrieved_text = "\\n\\n".join(output_blocks)
    return generate_final_answer(query, retrieved_text)

def gitasan_response(prompt):
    answer = search_verses(prompt)
    return answer
'''

with open("model_script.py", "w") as f:
    f.write(model_script)


In [ ]:
# === 3. Write app.py ===
app_code = '''
import streamlit as st
from model_script import gitasan_response
import base64

# Set config
st.set_page_config(page_title="🕉️ GitaSan: The Bhagavad Gita Assistant", layout="centered")

# === Add background image ===
def add_background(image_file):
    with open(image_file, "rb") as f:
        data = f.read()
    encoded = base64.b64encode(data).decode()
    st.markdown(
        f"""
        <style>
        [data-testid="stAppViewContainer"] {{
            background-image: url("data:image/png;base64,{encoded}");
            background-size: cover;
            background-position: left top;
            background-repeat: no-repeat;
            background-attachment: fixed;
        }}

        [data-testid="stHeader"] {{
            background: rgba(0,0,0,0);
        }}

        [data-testid="stToolbar"] {{
            right: 2rem;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

add_background("krishna_bg.png")  # <-- Make sure this file is in the same directory

# === App content ===
st.markdown("<h1 style='text-align: center; color: #FFFFFF;'>🕉️ GitaGPT</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center; font-size: 18px;'>Your spiritual guide powered by the Bhagavad Gita ✨</p>", unsafe_allow_html=True)

with st.form("chat_form", clear_on_submit=True):
    prompt = st.text_input("Ask a question about the Bhagavad Gita", "")
    submitted = st.form_submit_button("Send")

if submitted and prompt:
    with st.spinner("Meditating on your question... 🧘‍♂️"):
        response = gitasan_response(prompt)
        st.markdown("### 🙋 You asked:")
        st.write(prompt)
        st.markdown("### 🤖 GitaSan says:")
        st.write(response)

st.markdown("<hr><center><sub>Made with ❤️ and devotion</sub></center>", unsafe_allow_html=True)
'''

with open("app.py", "w") as f:
    f.write(app_code)



In [ ]:
# === 4. Launch Streamlit with ngrok ===
from pyngrok import ngrok
import threading, time, os

ngrok.kill()  # Clean up previous tunnels

# Launch Streamlit app
def run():
    os.system("streamlit run app.py")

threading.Thread(target=run).start()
time.sleep(5)

# Add your Ngrok token (if not already saved)
!ngrok config add-authtoken "2vO1dUWb2d63hrSjzhoV3mdkYC0_5UrYk4aMxhFBbDbcPY7QF"

# Create tunnel
public_url = ngrok.connect(8501, "http")
print("🌐 Your GitaSan is live at:", public_url)


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Your GitaSan is live at: NgrokTunnel: "https://ab0a-104-196-43-90.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
from PIL import Image
Image.open("krishna_bg.png").format


'PNG'